In [1]:
import pandas as pd
df15 = pd.read_csv('/content/NSE_BANKNIFTY24FEBFUT_15MIN.csv')
df5=pd.read_csv('/content/NSE_BANKNIFTY24FEBFUT_5MIN.csv')

In [2]:
df5.head()

,datetime,open,high,low,close,volume
0,2024-02-06 09:15:00,45959.6,46097.00,45922.8,45941.3,98460
1,2024-02-06 09:20:00,45940.5,45978.80,45869.0,45954.1,97095
2,2024-02-06 09:25:00,45954.1,45984.85,45901.0,45910.0,50340
3,2024-02-06 09:30:00,45910.0,45992.20,45892.0,45944.6,41895
4,2024-02-06 09:35:00,45944.6,46034.95,45920.0,46018.1,36705


In [3]:
df15.head()

,datetime,open,high,low,close,volume
0,2024-02-06 09:15:00,45959.6,46097.00,45869.0,45910.0,245895
1,2024-02-06 09:30:00,45910.0,46034.95,45892.0,45994.7,95505
2,2024-02-06 09:45:00,45993.6,46072.25,45962.4,46020.6,112320
3,2024-02-06 10:00:00,46022.4,46068.85,45990.0,45994.9,69405
4,2024-02-06 10:15:00,45999.9,46009.50,45880.0,45884.0,87405


In [4]:
df5['ema_10'] = df5['close'].ewm(span=10).mean()
df5['ema_20'] = df5['close'].ewm(span=20).mean()
df15['ema_10'] = df15['close'].ewm(span=10).mean()
df15['ema_20'] = df15['close'].ewm(span=20).mean()

In [5]:
df5.head()

,datetime,open,high,low,close,volume,ema_10,ema_20
0,2024-02-06 09:15:00,45959.6,46097.00,45922.8,45941.3,98460,45941.300000,45941.300000
1,2024-02-06 09:20:00,45940.5,45978.80,45869.0,45954.1,97095,45948.340000,45948.020000
2,2024-02-06 09:25:00,45954.1,45984.85,45901.0,45910.0,50340,45932.927575,45934.059284
3,2024-02-06 09:30:00,45910.0,45992.20,45892.0,45944.6,41895,45936.773119,45937.102226
4,2024-02-06 09:35:00,45944.6,46034.95,45920.0,46018.1,36705,45960.119855,45956.694898


In [6]:
df15.head()

,datetime,open,high,low,close,volume,ema_10,ema_20
0,2024-02-06 09:15:00,45959.6,46097.00,45869.0,45910.0,245895,45910.000000,45910.000000
1,2024-02-06 09:30:00,45910.0,46034.95,45892.0,45994.7,95505,45956.585000,45954.467500
2,2024-02-06 09:45:00,45993.6,46072.25,45962.4,46020.6,112320,45982.318605,45978.750958
3,2024-02-06 10:00:00,46022.4,46068.85,45990.0,45994.9,69405,45986.463614,45983.412936
4,2024-02-06 10:15:00,45999.9,46009.50,45880.0,45884.0,87405,45957.049097,45959.365793


In [7]:
filtered_df15 = df15[df15['ema_10'] > df15['ema_20']]

In [8]:
len(filtered_df15)

38

In [9]:
def get_date(datetime_str):
  return datetime_str.split(' ')[0]

In [10]:
df5['date'] = [get_date(x) for x in df5['datetime']]
trading_date_list_df5 = df5['date'].unique()
print(trading_date_list_df5)

['2024-02-06' '2024-02-07' '2024-02-08']


In [11]:
for date_str in trading_date_list_df5:
  filt = (date_str == df5['date'])
  days_high = df5.loc[filt, 'high'].max()
  df5.loc[filt, 'current_days_high'] = days_high

In [12]:
def get_previous_day(date_str, trading_date_list_df5):
    current_date = pd.Timestamp(date_str).date()
    for i in range(1,4):
        previous_day = current_date - pd.Timedelta(days=1)
        previous_day_str = previous_day.strftime('%Y-%m-%d')

        if previous_day in trading_date_list_df5:
            return previous_day
        else:
            current_date = previous_day
    return None

In [13]:
for date_str in trading_date_list_df5:
    unique_dates_list = pd.to_datetime(df5['date']).dt.date.unique()

    prev_day = get_previous_day(date_str, unique_dates_list)
    if prev_day is not None:
        prev_day_high = df5.loc[(df5['date'] == str(prev_day)), 'current_days_high'].values[0]
        df5.loc[df5['date'] == date_str, 'previous_day_high'] = prev_day_high

In [14]:
df5.head(5)

,datetime,open,high,low,close,volume,ema_10,ema_20,date,current_days_high,previous_day_high
0,2024-02-06 09:15:00,45959.6,46097.00,45922.8,45941.3,98460,45941.300000,45941.300000,2024-02-06,46165.0,NaN
1,2024-02-06 09:20:00,45940.5,45978.80,45869.0,45954.1,97095,45948.340000,45948.020000,2024-02-06,46165.0,NaN
2,2024-02-06 09:25:00,45954.1,45984.85,45901.0,45910.0,50340,45932.927575,45934.059284,2024-02-06,46165.0,NaN
3,2024-02-06 09:30:00,45910.0,45992.20,45892.0,45944.6,41895,45936.773119,45937.102226,2024-02-06,46165.0,NaN
4,2024-02-06 09:35:00,45944.6,46034.95,45920.0,46018.1,36705,45960.119855,45956.694898,2024-02-06,46165.0,NaN


In [15]:
cond2 = df5['close'] > df5['previous_day_high']
cond3 = df5['ema_10'] > df5['ema_20']
cond4 = df5['close'] > df5['ema_10']

filtered_df5 = df5[cond2 & cond3 & cond4]

In [16]:
filtered_df15['date'] = [get_date(x) for x in filtered_df15['datetime']]
filtered_df15_date_list = filtered_df15['date'].unique()
print(filtered_df15_date_list)

['2024-02-06' '2024-02-07' '2024-02-08']


<ipython-input-16-20e9e56c0af1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df15['date'] = [get_date(x) for x in filtered_df15['datetime']]


In [17]:
df15['date'] = [get_date(x) for x in df15['datetime']]
df15_date_list = df15['date'].unique()
print(df15_date_list)

['2024-02-06' '2024-02-07' '2024-02-08']


In [18]:
filtered_df15.head(30)

,datetime,open,high,low,close,volume,ema_10,ema_20,date
1,2024-02-06 09:30:00,45910.0,46034.95,45892.00,45994.7,95505,45956.585000,45954.467500,2024-02-06
2,2024-02-06 09:45:00,45993.6,46072.25,45962.40,46020.6,112320,45982.318605,45978.750958,2024-02-06
3,2024-02-06 10:00:00,46022.4,46068.85,45990.00,45994.9,69405,45986.463614,45983.412936,2024-02-06
10,2024-02-06 11:45:00,45958.0,46098.90,45930.00,46096.9,136260,45943.683396,45937.145221,2024-02-06
11,2024-02-06 12:00:00,46100.0,46165.00,46031.60,46078.9,204990,45970.699426,45956.456213,2024-02-06
12,2024-02-06 12:15:00,46070.2,46085.15,45916.00,45918.4,87780,45960.434657,45951.476024,2024-02-06
13,2024-02-06 12:30:00,45917.6,46036.60,45900.20,46022.6,63390,45972.461994,45960.463410,2024-02-06
14,2024-02-06 12:45:00,46015.6,46085.00,45946.40,45955.4,68310,45969.198984,45959.842899,2024-02-06
15,2024-02-06 13:00:00,45950.0,45999.00,45935.00,45993.7,35205,45973.840910,45963.881720,2024-02-06
16,2024-02-06 13:15:00,45993.7,46028.00,45955.10,46013.1,47520,45981.222480,45969.615081,2024-02-06


In [19]:
filtered_df5['date'] = [get_date(x) for x in filtered_df5['datetime']]
filtered_df5_date_list = filtered_df5['date'].unique()
print(filtered_df5_date_list)

['2024-02-07' '2024-02-08']


<ipython-input-19-f000ef683616>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df5['date'] = [get_date(x) for x in filtered_df5['datetime']]


In [20]:
df5['date'] = [get_date(x) for x in df5['datetime']]
df5_date_list = df5['date'].unique()
print(df5_date_list)

['2024-02-06' '2024-02-07' '2024-02-08']


In [21]:
filtered_df5.head(30)

,datetime,open,high,low,close,volume,ema_10,ema_20,date,current_days_high,previous_day_high
75,2024-02-07 09:15:00,46201.1,46281.75,46159.00,46265.8,170385,46016.812806,45987.085643,2024-02-07,46296.0,46165.0
76,2024-02-07 09:20:00,46260.0,46292.65,46215.00,46286.0,118440,46065.755941,46015.566492,2024-02-07,46296.0,46165.0
77,2024-02-07 09:25:00,46285.9,46296.00,46171.30,46209.4,80775,46091.873047,46034.034344,2024-02-07,46296.0,46165.0
78,2024-02-07 09:30:00,46201.1,46245.00,46193.95,46233.8,61245,46117.677951,46053.066654,2024-02-07,46296.0,46165.0
79,2024-02-07 09:35:00,46240.2,46252.75,46216.90,46243.4,38775,46140.536508,46071.199682,2024-02-07,46296.0,46165.0
80,2024-02-07 09:40:00,46240.8,46289.00,46139.75,46185.0,72210,46148.620780,46082.041076,2024-02-07,46296.0,46165.0
81,2024-02-07 09:45:00,46189.5,46205.00,46156.00,46193.1,31260,46156.707911,46092.621002,2024-02-07,46296.0,46165.0
82,2024-02-07 09:50:00,46193.1,46210.00,46121.45,46167.0,43845,46158.579200,46099.706465,2024-02-07,46296.0,46165.0
83,2024-02-07 09:55:00,46168.2,46189.95,46150.00,46181.9,24480,46162.819346,46107.536169,2024-02-07,46296.0,46165.0
149,2024-02-07 15:25:00,46118.1,46249.85,46112.70,46230.0,152820,46119.646262,46089.367652,2024-02-07,46296.0,46165.0


In [22]:
filtered_df15.date


1     2024-02-06
2     2024-02-06
3     2024-02-06
10    2024-02-06
11    2024-02-06
12    2024-02-06
13    2024-02-06
14    2024-02-06
15    2024-02-06
16    2024-02-06
17    2024-02-06
18    2024-02-06
19    2024-02-06
21    2024-02-06
24    2024-02-06
25    2024-02-07
26    2024-02-07
27    2024-02-07
28    2024-02-07
29    2024-02-07
30    2024-02-07
31    2024-02-07
32    2024-02-07
33    2024-02-07
34    2024-02-07
35    2024-02-07
45    2024-02-07
46    2024-02-07
47    2024-02-07
48    2024-02-07
49    2024-02-07
50    2024-02-08
51    2024-02-08
52    2024-02-08
53    2024-02-08
54    2024-02-08
55    2024-02-08
56    2024-02-08
Name: date, dtype: object

In [23]:
filtered_df5.date

75     2024-02-07
76     2024-02-07
77     2024-02-07
78     2024-02-07
79     2024-02-07
80     2024-02-07
81     2024-02-07
82     2024-02-07
83     2024-02-07
149    2024-02-07
152    2024-02-08
153    2024-02-08
155    2024-02-08
158    2024-02-08
159    2024-02-08
Name: date, dtype: object

In [24]:
type(filtered_df15['datetime'])

pandas.core.series.Series

In [25]:
type(df5.date)

pandas.core.series.Series

In [26]:
filtered_df15.head()

,datetime,open,high,low,close,volume,ema_10,ema_20,date
1,2024-02-06 09:30:00,45910.0,46034.95,45892.0,45994.7,95505,45956.585000,45954.467500,2024-02-06
2,2024-02-06 09:45:00,45993.6,46072.25,45962.4,46020.6,112320,45982.318605,45978.750958,2024-02-06
3,2024-02-06 10:00:00,46022.4,46068.85,45990.0,45994.9,69405,45986.463614,45983.412936,2024-02-06
10,2024-02-06 11:45:00,45958.0,46098.90,45930.0,46096.9,136260,45943.683396,45937.145221,2024-02-06
11,2024-02-06 12:00:00,46100.0,46165.00,46031.6,46078.9,204990,45970.699426,45956.456213,2024-02-06


In [27]:
from datetime import datetime

In [28]:

filtered_df15['datetime'] = pd.to_datetime(filtered_df15['datetime'])
filtered_df5['datetime'] = pd.to_datetime(filtered_df5['datetime'])

<ipython-input-28-610b1ea63c10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df15['datetime'] = pd.to_datetime(filtered_df15['datetime'])
<ipython-input-28-610b1ea63c10>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df5['datetime'] = pd.to_datetime(filtered_df5['datetime'])


In [29]:
type(filtered_df15['datetime'])

pandas.core.series.Series

In [30]:
filtered_df15.head()

,datetime,open,high,low,close,volume,ema_10,ema_20,date
1,2024-02-06 09:30:00,45910.0,46034.95,45892.0,45994.7,95505,45956.585000,45954.467500,2024-02-06
2,2024-02-06 09:45:00,45993.6,46072.25,45962.4,46020.6,112320,45982.318605,45978.750958,2024-02-06
3,2024-02-06 10:00:00,46022.4,46068.85,45990.0,45994.9,69405,45986.463614,45983.412936,2024-02-06
10,2024-02-06 11:45:00,45958.0,46098.90,45930.0,46096.9,136260,45943.683396,45937.145221,2024-02-06
11,2024-02-06 12:00:00,46100.0,46165.00,46031.6,46078.9,204990,45970.699426,45956.456213,2024-02-06


In [31]:
filtered_df15[filtered_df15['datetime']=='2023-12-29 11:45:00']

,datetime,open,high,low,close,volume,ema_10,ema_20,date


In [32]:
filtered_df15[filtered_df15['datetime']==(pd.Timestamp('2023-12-29 11:45:00') + pd.Timedelta(minutes=15))]

,datetime,open,high,low,close,volume,ema_10,ema_20,date


In [33]:
datetime_list_filtered_df15 = filtered_df15['datetime'].tolist()
len(datetime_list_filtered_df15)


38

In [34]:
datetime_list_filtered_df5 = filtered_df5['datetime'].tolist()
len(datetime_list_filtered_df5)

15

In [35]:
final_datetime_list = []

datetime_set_filtered_df15 = set(datetime_list_filtered_df15)
for dt in datetime_list_filtered_df5:
    if dt in datetime_set_filtered_df15 or (dt - pd.Timedelta(minutes=5)) in datetime_set_filtered_df15 or (dt - pd.Timedelta(minutes=10)) in datetime_set_filtered_df15:
        final_datetime_list.append(dt)

print(final_datetime_list)


[Timestamp('2024-02-07 09:15:00'), Timestamp('2024-02-07 09:20:00'), Timestamp('2024-02-07 09:25:00'), Timestamp('2024-02-07 09:30:00'), Timestamp('2024-02-07 09:35:00'), Timestamp('2024-02-07 09:40:00'), Timestamp('2024-02-07 09:45:00'), Timestamp('2024-02-07 09:50:00'), Timestamp('2024-02-07 09:55:00'), Timestamp('2024-02-07 15:25:00'), Timestamp('2024-02-08 09:25:00'), Timestamp('2024-02-08 09:30:00'), Timestamp('2024-02-08 09:40:00'), Timestamp('2024-02-08 09:55:00'), Timestamp('2024-02-08 10:00:00')]


In [36]:
len(final_datetime_list)

15

In [37]:
len(datetime_set_filtered_df15)

38

In [38]:
ce = filtered_df5[filtered_df5['datetime'].isin(final_datetime_list)]

In [39]:
ce.shape

(15, 11)

In [40]:
ce.head(15)

,datetime,open,high,low,close,volume,ema_10,ema_20,date,current_days_high,previous_day_high
75,2024-02-07 09:15:00,46201.1,46281.75,46159.00,46265.8,170385,46016.812806,45987.085643,2024-02-07,46296.0,46165.0
76,2024-02-07 09:20:00,46260.0,46292.65,46215.00,46286.0,118440,46065.755941,46015.566492,2024-02-07,46296.0,46165.0
77,2024-02-07 09:25:00,46285.9,46296.00,46171.30,46209.4,80775,46091.873047,46034.034344,2024-02-07,46296.0,46165.0
78,2024-02-07 09:30:00,46201.1,46245.00,46193.95,46233.8,61245,46117.677951,46053.066654,2024-02-07,46296.0,46165.0
79,2024-02-07 09:35:00,46240.2,46252.75,46216.90,46243.4,38775,46140.536508,46071.199682,2024-02-07,46296.0,46165.0
80,2024-02-07 09:40:00,46240.8,46289.00,46139.75,46185.0,72210,46148.620780,46082.041076,2024-02-07,46296.0,46165.0
81,2024-02-07 09:45:00,46189.5,46205.00,46156.00,46193.1,31260,46156.707911,46092.621002,2024-02-07,46296.0,46165.0
82,2024-02-07 09:50:00,46193.1,46210.00,46121.45,46167.0,43845,46158.579200,46099.706465,2024-02-07,46296.0,46165.0
83,2024-02-07 09:55:00,46168.2,46189.95,46150.00,46181.9,24480,46162.819346,46107.536169,2024-02-07,46296.0,46165.0
149,2024-02-07 15:25:00,46118.1,46249.85,46112.70,46230.0,152820,46119.646262,46089.367652,2024-02-07,46296.0,46165.0


In [41]:
start_time = pd.to_datetime("9:45:00").time()
stop_time = pd.to_datetime("15:00:00").time()

# Filter rows based on conditions
filtered_ce = ce[~((ce['datetime'].dt.time < start_time) | (ce['datetime'].dt.time > stop_time))]

In [42]:
filtered_ce

,datetime,open,high,low,close,volume,ema_10,ema_20,date,current_days_high,previous_day_high
81,2024-02-07 09:45:00,46189.5,46205.00,46156.00,46193.1,31260,46156.707911,46092.621002,2024-02-07,46296.0,46165.0
82,2024-02-07 09:50:00,46193.1,46210.00,46121.45,46167.0,43845,46158.579200,46099.706465,2024-02-07,46296.0,46165.0
83,2024-02-07 09:55:00,46168.2,46189.95,46150.00,46181.9,24480,46162.819346,46107.536169,2024-02-07,46296.0,46165.0
158,2024-02-08 09:55:00,46273.9,46342.70,46264.30,46300.9,45900,46265.118543,46210.547712,2024-02-08,46475.7,46296.0
159,2024-02-08 10:00:00,46296.1,46410.00,46275.05,46410.0,79335,46291.460626,46229.543170,2024-02-08,46475.7,46296.0


In [43]:
ce['strike'] = (ce['close'].astype(float) / 100).apply(lambda x: int(round(x) * 100 - 200))


In [44]:
ce

,datetime,open,high,low,close,volume,ema_10,ema_20,date,current_days_high,previous_day_high,strike
75,2024-02-07 09:15:00,46201.1,46281.75,46159.00,46265.8,170385,46016.812806,45987.085643,2024-02-07,46296.0,46165.0,46100
76,2024-02-07 09:20:00,46260.0,46292.65,46215.00,46286.0,118440,46065.755941,46015.566492,2024-02-07,46296.0,46165.0,46100
77,2024-02-07 09:25:00,46285.9,46296.00,46171.30,46209.4,80775,46091.873047,46034.034344,2024-02-07,46296.0,46165.0,46000
78,2024-02-07 09:30:00,46201.1,46245.00,46193.95,46233.8,61245,46117.677951,46053.066654,2024-02-07,46296.0,46165.0,46000
79,2024-02-07 09:35:00,46240.2,46252.75,46216.90,46243.4,38775,46140.536508,46071.199682,2024-02-07,46296.0,46165.0,46000
80,2024-02-07 09:40:00,46240.8,46289.00,46139.75,46185.0,72210,46148.620780,46082.041076,2024-02-07,46296.0,46165.0,46000
81,2024-02-07 09:45:00,46189.5,46205.00,46156.00,46193.1,31260,46156.707911,46092.621002,2024-02-07,46296.0,46165.0,46000
82,2024-02-07 09:50:00,46193.1,46210.00,46121.45,46167.0,43845,46158.579200,46099.706465,2024-02-07,46296.0,46165.0,46000
83,2024-02-07 09:55:00,46168.2,46189.95,46150.00,46181.9,24480,46162.819346,46107.536169,2024-02-07,46296.0,46165.0,46000
149,2024-02-07 15:25:00,46118.1,46249.85,46112.70,46230.0,152820,46119.646262,46089.367652,2024-02-07,46296.0,46165.0,46000


In [45]:
# Assuming 'CE' is the DataFrame containing the rows of 'CE' type
ce['symbol'] = "NSE:BANKNIFTY24FEB" + ce['strike'].astype(str) + "CE"


In [46]:
ce

,datetime,open,high,low,close,volume,ema_10,ema_20,date,current_days_high,previous_day_high,strike,symbol
75,2024-02-07 09:15:00,46201.1,46281.75,46159.00,46265.8,170385,46016.812806,45987.085643,2024-02-07,46296.0,46165.0,46100,NSE:BANKNIFTY24FEB46100CE
76,2024-02-07 09:20:00,46260.0,46292.65,46215.00,46286.0,118440,46065.755941,46015.566492,2024-02-07,46296.0,46165.0,46100,NSE:BANKNIFTY24FEB46100CE
77,2024-02-07 09:25:00,46285.9,46296.00,46171.30,46209.4,80775,46091.873047,46034.034344,2024-02-07,46296.0,46165.0,46000,NSE:BANKNIFTY24FEB46000CE
78,2024-02-07 09:30:00,46201.1,46245.00,46193.95,46233.8,61245,46117.677951,46053.066654,2024-02-07,46296.0,46165.0,46000,NSE:BANKNIFTY24FEB46000CE
79,2024-02-07 09:35:00,46240.2,46252.75,46216.90,46243.4,38775,46140.536508,46071.199682,2024-02-07,46296.0,46165.0,46000,NSE:BANKNIFTY24FEB46000CE
80,2024-02-07 09:40:00,46240.8,46289.00,46139.75,46185.0,72210,46148.620780,46082.041076,2024-02-07,46296.0,46165.0,46000,NSE:BANKNIFTY24FEB46000CE
81,2024-02-07 09:45:00,46189.5,46205.00,46156.00,46193.1,31260,46156.707911,46092.621002,2024-02-07,46296.0,46165.0,46000,NSE:BANKNIFTY24FEB46000CE
82,2024-02-07 09:50:00,46193.1,46210.00,46121.45,46167.0,43845,46158.579200,46099.706465,2024-02-07,46296.0,46165.0,46000,NSE:BANKNIFTY24FEB46000CE
83,2024-02-07 09:55:00,46168.2,46189.95,46150.00,46181.9,24480,46162.819346,46107.536169,2024-02-07,46296.0,46165.0,46000,NSE:BANKNIFTY24FEB46000CE
149,2024-02-07 15:25:00,46118.1,46249.85,46112.70,46230.0,152820,46119.646262,46089.367652,2024-02-07,46296.0,46165.0,46000,NSE:BANKNIFTY24FEB46000CE


In [47]:
unique_symbols = ce['symbol'].unique()
unique_symbols

array(['NSE:BANKNIFTY24FEB46100CE', 'NSE:BANKNIFTY24FEB46000CE',
       'NSE:BANKNIFTY24FEB46200CE'], dtype=object)

In [48]:
filtered_df15_pe = df15[df15['ema_10'] < df15['ema_20']]

In [49]:
for date_str in trading_date_list_df5:
  filt = (date_str == df5['date'])
  days_low = df5.loc[filt, 'low'].min()
  df5.loc[filt, 'current_days_low'] = days_low

In [50]:
for date_str in trading_date_list_df5:
    unique_dates_list = pd.to_datetime(df5['date']).dt.date.unique()

    prev_day = get_previous_day(date_str, unique_dates_list)
    if prev_day is not None:
        prev_day_low = df5.loc[(df5['date'] == str(prev_day)), 'current_days_low'].values[0]
        df5.loc[df5['date'] == date_str, 'previous_day_low'] = prev_day_low

In [51]:
cond2 = df5['close'] < df5['previous_day_low']
cond3 = df5['ema_10'] < df5['ema_20']
cond4 = df5['close'] < df5['ema_10']

filtered_df5_pe = df5[cond2 & cond3 & cond4]

In [52]:
filtered_df15_pe['datetime'] = pd.to_datetime(filtered_df15_pe['datetime'])
filtered_df5_pe['datetime'] = pd.to_datetime(filtered_df5_pe['datetime'])

<ipython-input-52-79d6d9988f5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df15_pe['datetime'] = pd.to_datetime(filtered_df15_pe['datetime'])
<ipython-input-52-79d6d9988f5a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df5_pe['datetime'] = pd.to_datetime(filtered_df5_pe['datetime'])


In [53]:
datetime_list_filtered_df15_pe = filtered_df15_pe['datetime'].tolist()
datetime_list_filtered_df5_pe = filtered_df5_pe['datetime'].tolist()

In [54]:
final_datetime_list_pe = []

datetime_set_filtered_df15_pe = set(datetime_list_filtered_df15_pe)
for dt in datetime_list_filtered_df5_pe:
    if dt in datetime_set_filtered_df15_pe or (dt - pd.Timedelta(minutes=5)) in datetime_set_filtered_df15_pe or (dt - pd.Timedelta(minutes=10)) in datetime_set_filtered_df15_pe:
        final_datetime_list_pe.append(dt)

print(final_datetime_list_pe)

[Timestamp('2024-02-08 11:00:00'), Timestamp('2024-02-08 11:05:00'), Timestamp('2024-02-08 11:10:00'), Timestamp('2024-02-08 11:15:00'), Timestamp('2024-02-08 11:20:00'), Timestamp('2024-02-08 11:25:00'), Timestamp('2024-02-08 11:30:00'), Timestamp('2024-02-08 11:35:00'), Timestamp('2024-02-08 11:40:00'), Timestamp('2024-02-08 11:45:00'), Timestamp('2024-02-08 11:50:00'), Timestamp('2024-02-08 11:55:00'), Timestamp('2024-02-08 12:15:00'), Timestamp('2024-02-08 12:20:00'), Timestamp('2024-02-08 12:25:00'), Timestamp('2024-02-08 12:30:00'), Timestamp('2024-02-08 12:35:00'), Timestamp('2024-02-08 13:00:00'), Timestamp('2024-02-08 13:20:00'), Timestamp('2024-02-08 13:35:00'), Timestamp('2024-02-08 13:45:00'), Timestamp('2024-02-08 13:50:00'), Timestamp('2024-02-08 14:00:00'), Timestamp('2024-02-08 14:05:00'), Timestamp('2024-02-08 14:10:00'), Timestamp('2024-02-08 14:20:00'), Timestamp('2024-02-08 14:25:00'), Timestamp('2024-02-08 14:30:00'), Timestamp('2024-02-08 14:35:00'), Timestamp('20

In [55]:
pe = filtered_df5_pe[filtered_df5_pe['datetime'].isin(final_datetime_list_pe)]

In [56]:
pe['strike'] = (pe['close'].astype(float) / 100).apply(lambda x: int(round(x) * 100 + 200))



<ipython-input-56-fa2f2876a47a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pe['strike'] = (pe['close'].astype(float) / 100).apply(lambda x: int(round(x) * 100 + 200))


In [57]:
pe.head()

,datetime,open,high,low,close,volume,ema_10,ema_20,date,current_days_high,previous_day_high,current_days_low,previous_day_low,strike
171,2024-02-08 11:00:00,45881.9,45881.95,45530.00,45572.2,380175,46047.785911,46129.190469,2024-02-08,46475.7,46296.0,45053.2,45901.0,45800
172,2024-02-08 11:05:00,45571.9,45650.00,45463.20,45573.4,190110,45961.533927,46076.258042,2024-02-08,46475.7,46296.0,45053.2,45901.0,45800
173,2024-02-08 11:10:00,45578.9,45685.00,45564.80,45637.6,91920,45902.636850,46034.481084,2024-02-08,46475.7,46296.0,45053.2,45901.0,45800
174,2024-02-08 11:15:00,45640.0,45675.00,45592.55,45625.0,63510,45852.157422,45995.482885,2024-02-08,46475.7,46296.0,45053.2,45901.0,45800
175,2024-02-08 11:20:00,45625.8,45647.25,45582.60,45632.9,53730,45812.292436,45960.951181,2024-02-08,46475.7,46296.0,45053.2,45901.0,45800


In [58]:

filtered_pe = pe[~((pe['datetime'].dt.time < start_time) | (pe['datetime'].dt.time > stop_time))]

In [59]:

pe['symbol'] = "NSE:BANKNIFTY24FEB" + pe['strike'].astype(str) + "PE"

<ipython-input-59-3cb277057635>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pe['symbol'] = "NSE:BANKNIFTY24FEB" + pe['strike'].astype(str) + "PE"


In [60]:
unique_symbols_pe = pe['symbol'].unique()
unique_symbols_pe

array(['NSE:BANKNIFTY24FEB45800PE', 'NSE:BANKNIFTY24FEB45900PE',
       'NSE:BANKNIFTY24FEB45700PE', 'NSE:BANKNIFTY24FEB45600PE',
       'NSE:BANKNIFTY24FEB45500PE', 'NSE:BANKNIFTY24FEB45400PE'],
      dtype=object)